In [1]:
import imageio # Used for reading in the images.
import glob # Used for browsing through the directories.
from tqdm.notebook import tqdm_notebook # Used for tracking progress.
from skimage.filters import threshold_multiotsu # Used for identifying thresholds for image segmentation.

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

# Data directories.
# We enter here some intrinsic details of the dataset needed for our segmentation script.
# Set the variable "base_data_dir_str" to the path where the dataset is stored on your own workstation.
base_data_dir_str = '/export/scratch2/mbk/2DeteCT_recons/' 

# User defined settings.

# Select the ID(s) of the slice(s) you want to segment.
# Adjust this to your liking.
# slice_id = range(43,44) #random.sample(range(1, 6370+1), 50)
slice_id = range(0, 6370+1)

# Nomenclatura.
recon_name = 'reconstruction.tif'
segm_name = 'segmentation.tif'

# Get the sorted list of all mode2 reconstructions in the dataset.
list_recons = sorted(glob.glob(base_data_dir_str + '/*/mode2/' + recon_name))


# Helper functions.
def apply_thresholds(img, thresholds):
    segm_img = np.zeros(shape=img.shape)
    for i, t in enumerate(thresholds):
        segm_img[img > t] = i + 1
    return segm_img

def cylinder(r, n, x0, y0):
    cyl = np.zeros(shape=(n,n),dtype=np.bool)
    x = np.linspace(-1,1,1024)
    y = np.linspace(-1,1,1024)
    x,y = np.meshgrid(x,y)
    cyl = (x-x0/n)**2 + (y-y0/n)**2 < (r/n)**2
    return cyl

def cyl_finder(center, img):
    n = 1024
    r2 = 878 # Diameter of inner tube circle.
    r1 = 912 # Diameter of outer tube circle.
    tube_mask = 1*(cylinder(r1, n, center[0], center[1]) + -1 *(cylinder(r2, n, center[0], center[1]) - 1))
    metric = -np.sum(tube_mask*img)
    return metric 

def tube_maker(center):
    n = 1024
    r2 = 878 # Diameter of inner tube circle.
    r1 = 912 # Diameter of outer tube circle.
    tube_mask = 1*(cylinder(r1, n, center[0], center[1]) + -1 *cylinder(r2, n, center[0], center[1]))
    return tube_mask


# Main segmentation function.
def segment_2DeteCT(path):
    # The four class segmentation follows in 6 steps.
    # Step 1: Find the tube wall via a fixed mask - matching.
    # Step 2: Define the background as everything outside the tube wall.
    # Step 3: Find the objects and tube wall via multiotsu.
    # Step 4: Distinguish the objects by removing the tube wall and the background from the multiotsu segmentation.
    # Step 5: Find the filler material by removing all other classes from the whole image.
    # Step 6: Put together final four class segmentation.
    
    # Read in the image.
    img = imageio.imread(path)
    
    # Step 1: Find the tube wall via a fixed mask - matching.
    # Basic information about tube wall and image size.
    n = 1024
    r2 = 878 # Diameter of inner tube circle.
    r1 = 912 # Diameter of outer tube circle.
    
    result = sp.optimize.minimize(cyl_finder, (1,1), args=img, method='Nelder-Mead').x
    x0 = round(result[0])
    y0 = round(result[1])
    #print('Center of tube wall lies at: (',x0,',',y0,')')
    
    tube = tube_maker((x0,y0))
    tube_cleaned = np.clip(tube,0,1)
    
    # Step 2: Find the background via inversion and diameter_opening.
    background = -1*(cylinder(r1, n, x0, y0)-1) # Define background as everything outside the tube_wall.
    background = background.astype('bool')
    
    # Step 3: Find the objects and tube wall via multiotsu.
    bins = 256
    
    # thresholds = threshold_multiotsu(img, classes=3, nbins=bins) # NxN x bins ^ nr_classes
    # print('The thresholds for the multiotsu segmentaion are:', thresholds)
    # Thresholds found through method above are on average [0.00110607 0.00407358].
    
    # Manually defined for all slices.
    thresholds = [0.00110607, 0.00407358]
    
    # Apply the thresholds for the segmentaion.
    segm_img = apply_thresholds(img, thresholds)
    segm_img = np.clip(segm_img,0,1)
    segm_img = segm_img.astype('bool')
    
    # Step 4: Distinguish the objects by removing the tube wall and the background from the multiotsu segmentation.
    objects = segm_img + (-1)*(tube_cleaned+background)
    objects_cleaned = np.clip(objects,0,1)
    objects_cleaned = objects_cleaned.astype('bool')
    
    # Step 5: Find the filler material by removing all other classes from the whole image.
    classified = background + objects_cleaned + tube_cleaned
    classified_bool = classified.astype('bool')
    powder = ~classified_bool
    
    # Check if they overlap.
    test1 = background.astype('int')+tube_cleaned.astype('int')+objects_cleaned.astype('int')+powder.astype('int')
    assert np.amax(test1) == 1, str("Classes overlap in the segmentation of: " + path)
    
    # Check if there is something not classified.
    test2 = background+tube_cleaned+objects_cleaned+powder
    assert np.unique(test2) == [1], str("There are unclassified pixels in the segmentation of: " + path)
    
    # Step 6: Put together final four class segmentation.
    segmentation = 1*background.astype('uint8')+2*tube_cleaned.astype('uint8')+3*powder.astype('uint8')+4*objects_cleaned.astype('uint8')
    
    # Save segmentation.
    folder_name = path.rsplit('reconstruction.tif')[0]
    imageio.imwrite(str(folder_name+'/segmentation.tif'), segmentation.astype(np.uint8))
    
    return None

# Main.
for i_slc in tqdm_notebook(slice_id, desc = 'Loop over all desired slices in the dataset'):
    
    segment_2DeteCT(list_recons[i_slc])
    

Loop over all desired slices in the dataset:   0%|          | 0/6371 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

